In [1]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm
import shutil

from architect_graph import Architect
from model_search import Network

In [2]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "arch_weight_decay": 1e-3,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate_min": 0.001,
    "learning_rate": 0.025,
    "unrolled": True,
    "epochs": 10,
    "batch_size": 4,
    "save": "EXP"
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

In [3]:
np_ds_train = (np.random.randint(0, 256, (20, 4, 4, 3)).astype(np.float32), np.random.randint(0, 2, (20, 4, 4, 1)).astype(np.float32))
np_ds_valid = (np.random.randint(0, 256, (20, 4, 4, 3)).astype(np.float32), np.random.randint(0, 2, (20, 4, 4, 1)).astype(np.float32))
ds_train = tf.data.Dataset.from_tensor_slices(np_ds_train).batch(args.batch_size)
ds_valid = tf.data.Dataset.from_tensor_slices(np_ds_valid).batch(args.batch_size)

In [14]:
num_iterations = int(np_ds_train[0].shape[0] / args.batch_size)
criterion = tf.losses.sigmoid_cross_entropy
model = Network(3, 3, criterion)
optimizer = tf.train.MomentumOptimizer(args.learning_rate_min, args.momentum)
init_l = tf.local_variables_initializer()

In [5]:
train_it = ds_train.make_one_shot_iterator()
valid_it = ds_valid.make_one_shot_iterator()
x_train, y_train = train_it.get_next()
x_valid, y_valid = valid_it.get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [6]:
logits = model(x_train)
architect = Architect(model, args)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [8]:
with tf.control_dependencies([architect_step]):
    w_var = model.get_thetas()
    loss = model._loss(logits, y_train)
    grads = tf.gradients(loss, w_var)
    clipped_gradients, norm = tf.clip_by_global_norm(grads, args.grad_clip)
    opt_op = optimizer.apply_gradients(zip(clipped_gradients, w_var))
acc = tf.metrics.accuracy(predictions=logits, labels=y_train)
miou = tf.metrics.mean_iou(predictions=logits, labels=y_train, num_classes=2)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [ ]:
with tf.Session() as sess:
    sess.run(init_l)
    sess.run(tf.global_variables_initializer())
    sess.run(architect_step)
    sess.run(opt_op)
    l, accuracy, mean_iou = sess.run([loss, acc, iou])